In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
import time

import tensorflow as tf
tf.random.set_seed(42)

def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / max(y_true))) * 100

def RMSELoss(yhat,y):
    return torch.sqrt(torch.mean((yhat-y)**2))

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
# Specifying the path of file
path= r"C:\Users\sashah8\OneDrive - North Carolina State University\Desktop\Q4\Data\L3\data_cleaned"

# Listing just the required files
fileList = os.listdir(path)
fileList.remove("Date.csv")

# Previewing the required file names
print(fileList)

['AT.csv', 'BE.csv', 'BG.csv', 'CH.csv', 'CZ.csv', 'DK.csv', 'ES.csv', 'FR.csv', 'GR.csv', 'IT.csv', 'NL.csv', 'PT.csv', 'SI.csv', 'SK.csv']


In [3]:
# Just for previewing the columns
pd.read_csv(os.path.join(path, fileList[2]))

,Demand,Normalized_net,Temperature,DNI,DHI
0,0.0,0.071032,-13.350,0.0,0.0
1,4585.0,0.624909,-13.594,0.0,0.0
2,4440.0,0.607393,-13.781,0.0,0.0
3,4418.0,0.604735,-14.042,0.0,0.0
4,4393.0,0.601715,-14.340,0.0,0.0
...,...,...,...,...,...
43819,4760.0,0.646050,-1.351,0.0,0.0
43820,4613.0,0.628292,-1.009,0.0,0.0
43821,4558.0,0.621648,-0.986,0.0,0.0
43822,4410.0,0.603769,-1.270,0.0,0.0


### PROCESSING THE DATA

In [4]:
# Restricting the load data to a single zone
fileName  = fileList[2]

zoneData = pd.read_csv(os.path.join(path, fileName), usecols= ["Demand", "Temperature"])
zoneData.columns = ["Load_"+str(fileName[:-4]), "Temp_"+str(fileName[:-4])]

# For fixing one missing row
# zoneData = pd.concat([zoneData.loc[[0]], zoneData], axis= 0).reset_index(drop= True)

zoneData

,Load_BG,Temp_BG
0,0.0,-13.350
1,4585.0,-13.594
2,4440.0,-13.781
3,4418.0,-14.042
4,4393.0,-14.340
...,...,...
43819,4760.0,-1.351
43820,4613.0,-1.009
43821,4558.0,-0.986
43822,4410.0,-1.270


### FEATURE ENGINEERING AND SPLITTING INTO TRAIN-TEST SPLIT

In [5]:
def lagGenerator(dF, lagList):
    dF_temp = dF.filter(regex= "Load").copy()
    
    for i in lagList:
        dF_temp.loc[:, "Lag-" + str(i)] = dF_temp.iloc[:,[0]].shift(i*24)
    
    return(dF_temp.filter(regex= "Lag"))

In [6]:
Load_lags = lagGenerator(dF=zoneData, lagList=[2,7])
Load_lags.head(2)

,Lag-2,Lag-7
0,NaN,NaN
1,NaN,NaN


In [7]:
dataDf = pd.concat([zoneData, Load_lags], axis= 1).dropna(axis= 0).reset_index(drop= True)
dataDf

,Load_BG,Temp_BG,Lag-2,Lag-7
0,4987.0,-12.111,4532.0,0.0
1,4918.0,-12.294,4487.0,4585.0
2,4964.0,-12.553,4519.0,4440.0
3,5081.0,-12.860,4678.0,4418.0
4,5570.0,-13.264,5190.0,4393.0
...,...,...,...,...
43651,4760.0,-1.351,5053.0,4146.0
43652,4613.0,-1.009,4956.0,4108.0
43653,4558.0,-0.986,4631.0,3964.0
43654,4410.0,-1.270,4520.0,3734.0


In [8]:
# Setting the train-test split ratio
ratio = 0.1

In [9]:
Num_train, Num_test = int(len(dataDf)/24 * (1 - ratio)) * 24, int(len(dataDf)/24 * ratio) * 24

In [10]:
# Splitting the data into train and test
data_train, data_test = dataDf[:Num_train], dataDf[Num_train:]

In [11]:
Num_train, Num_valid = int(len(data_train)/24 * (1 - ratio)) * 24, int(len(data_train)/24 * ratio) * 24

In [12]:
# Splitting the data into train and test
data_train, data_valid = data_train[:Num_train], data_train[Num_train:]

### SPLITTING INTO X AND Y PART WITH SEQUENCE BATCHING

In [13]:
X_train_df, Y_train_df = data_train.iloc[:, 1:], data_train.iloc[:, :1]

X_valid_df, Y_valid_df = data_valid.iloc[:, 1:], data_valid.iloc[:, :1]

X_test_df, Y_test_df = data_test.iloc[:, 1:], data_test.iloc[:, :1]

In [14]:
# Preparing the scaler
ValScaler = MinMaxScaler()

X_train = ValScaler.fit_transform(X_train_df)
X_valid = ValScaler.transform(X_valid_df)
X_test = ValScaler.transform(X_test_df)

Y_train = ValScaler.fit_transform(Y_train_df)
Y_valid = ValScaler.transform(Y_valid_df)
Y_test = ValScaler.transform(Y_test_df)

In [15]:
def splitData(dataset):
    data = np.array(np.split(dataset, len(dataset)/24))
    return data

In [16]:
def supervisedSet(X_part, Y_part, steps= 24):
    X_array, Y_array = np.array(X_part), np.array(Y_part)
    X, Y = [], []
    
    for i in range(len(X_array)):
        endIdx = i + steps
        
        if endIdx > len(X_array):
            break
        
        batch_X, batch_Y = X_array[i:endIdx], Y_array[i:endIdx]
        
        X.append(batch_X)
        Y.append(batch_Y)
        
    return np.array(X), np.array(Y)

In [17]:
X_train, Y_train = supervisedSet(X_train, Y_train)

X_valid, Y_valid = splitData(X_valid), splitData(Y_valid)

X_test, Y_test = splitData(X_test), splitData(Y_test)

### WORKING ON THE MODEL

In [18]:
epochs = 100
batchSize = 32

In [19]:
# Encoder-decoder architecture [tanh required to use tensorflow on gpu]
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units= 128, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2]), dropout= 0.2),
    tf.keras.layers.RepeatVector(Y_train.shape[1]),
    tf.keras.layers.LSTM(units= 128, activation= "tanh", return_sequences= True, dropout= 0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 128, activation= "relu")),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 1))
])

model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [20]:
# model = tf.keras.Sequential([
#     tf.keras.layers.LSTM(units= 256, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2])),
#     tf.keras.layers.Dense(units= 128, activation= "tanh"),
#     tf.keras.layers.Dense(units= 1)
# ])

# model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [21]:
# Storing the start time of model fitting process
startTime = time.time()

In [22]:
model.fit(X_train, Y_train, epochs= epochs, batch_size= batchSize, verbose= 1, shuffle= True, validation_data= (X_valid, Y_valid))

Epoch 1/100
1105/1105 [==============================] - 15s 9ms/step - loss: 0.0087 - mape: 15646.9824 - mae: 0.0702 - val_loss: 0.0041 - val_mape: 9.3611 - val_mae: 0.0535
Epoch 2/100
1105/1105 [==============================] - 9s 9ms/step - loss: 0.0060 - mape: 15040.3291 - mae: 0.0611 - val_loss: 0.0032 - val_mape: 8.7199 - val_mae: 0.0467
Epoch 3/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0037 - mape: 14317.0996 - mae: 0.0455 - val_loss: 0.0027 - val_mape: 7.4936 - val_mae: 0.0423
Epoch 4/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0026 - mape: 13742.3389 - mae: 0.0367 - val_loss: 0.0021 - val_mape: 6.1601 - val_mae: 0.0353
Epoch 5/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0022 - mape: 13638.4990 - mae: 0.0330 - val_loss: 0.0022 - val_mape: 5.9913 - val_mae: 0.0353
Epoch 6/100
1105/1105 [==============================] - 9s 9ms/step - loss: 0.0020 - mape: 13324.1807 - mae: 0.0310 - val_loss: 0.001

1105/1105 [==============================] - 9s 8ms/step - loss: 0.0011 - mape: 11824.4092 - mae: 0.0215 - val_loss: 0.0035 - val_mape: 7.9397 - val_mae: 0.0440
Epoch 49/100
1105/1105 [==============================] - 8s 7ms/step - loss: 0.0011 - mape: 11257.4941 - mae: 0.0215 - val_loss: 0.0023 - val_mape: 6.7668 - val_mae: 0.0389
Epoch 50/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0011 - mape: 10397.2705 - mae: 0.0215 - val_loss: 0.0028 - val_mape: 7.5703 - val_mae: 0.0443
Epoch 51/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0011 - mape: 10835.8271 - mae: 0.0218 - val_loss: 0.0023 - val_mape: 6.4395 - val_mae: 0.0381
Epoch 52/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0011 - mape: 11132.3994 - mae: 0.0212 - val_loss: 0.0018 - val_mape: 6.0709 - val_mae: 0.0339
Epoch 53/100
1105/1105 [==============================] - 9s 8ms/step - loss: 0.0011 - mape: 11507.0098 - mae: 0.0214 - val_loss: 0.0029 - val_

Epoch 95/100
1105/1105 [==============================] - 9s 8ms/step - loss: 8.5393e-04 - mape: 6599.9839 - mae: 0.0197 - val_loss: 0.0021 - val_mape: 6.6660 - val_mae: 0.0382
Epoch 96/100
1105/1105 [==============================] - 9s 8ms/step - loss: 8.8399e-04 - mape: 10215.4531 - mae: 0.0197 - val_loss: 0.0019 - val_mape: 6.1715 - val_mae: 0.0356
Epoch 97/100
1105/1105 [==============================] - 9s 8ms/step - loss: 8.3579e-04 - mape: 5859.3994 - mae: 0.0195 - val_loss: 0.0018 - val_mape: 6.0903 - val_mae: 0.0343
Epoch 98/100
1105/1105 [==============================] - 9s 8ms/step - loss: 7.8126e-04 - mape: 5295.5566 - mae: 0.0191 - val_loss: 0.0022 - val_mape: 6.4732 - val_mae: 0.0371
Epoch 99/100
1105/1105 [==============================] - 9s 8ms/step - loss: 7.7180e-04 - mape: 4406.9434 - mae: 0.0190 - val_loss: 0.0017 - val_mape: 5.7569 - val_mae: 0.0323
Epoch 100/100
1105/1105 [==============================] - 9s 8ms/step - loss: 7.6209e-04 - mape: 4758.1411 - mae:

In [23]:
calcExecTime = (time.time() - startTime)
print('Total execution time in seconds: ' + str(calcExecTime))

Total execution time in seconds: 852.8627743721008


### MAKING PREDICTIONS AND GENERATING RESULTS

In [24]:
predictions = model.predict(X_test)

6/6 [==============================] - 1s 7ms/step


In [25]:
predArr = predictions.reshape(predictions.shape[0] * predictions.shape[1], predictions.shape[2]).ravel()
predArr

array([0.40625325, 0.41254568, 0.42211902, ..., 0.5954155 , 0.58227384,
       0.57655215], dtype=float32)

In [26]:
Y_testArr = Y_test.reshape(Y_test.shape[0] * Y_test.shape[1], Y_test.shape[2]).ravel()
Y_testArr

array([0.45385359, 0.45796086, 0.4659338 , ..., 0.62164774, 0.60376903,
       0.58359507])

In [27]:
def inverse_transform(df):
    column_names = df.columns
    for col in column_names:
        df[col] = ValScaler.inverse_transform(df[[col]]) 
    return df

In [28]:
comparisionDf = pd.DataFrame(data= {"Values": Y_testArr, "Predictions": predArr})
comparisionDf = inverse_transform(comparisionDf)
comparisionDf

,Values,Predictions
0,3169.0,2774.964355
1,3203.0,2827.052979
2,3269.0,2906.301270
3,3521.0,3049.727051
4,3848.0,3184.976562
...,...,...
4363,4760.0,5001.333496
4364,4613.0,4589.671387
4365,4558.0,4340.849609
4366,4410.0,4232.062988


In [29]:
mape(comparisionDf["Values"], comparisionDf["Predictions"])

6.5205820428433965